In [ ]:
exp_dir = "F:\\Cambridge\\MENG\\UROP 2022\\Data\\SyMBac\\T100-400_iter20_lysis004\\"
mask_dir = exp_dir + "masks\\"
channel_image_dir = exp_dir + "convolutions\\"
lineage_dir = exp_dir + "truth\\"